In [1]:
# 사전 설치 : pip install fpdf
import os
import gradio as gr
from langchain.llms import Ollama
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from fpdf import FPDF

c:\aiproject2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Ollama 설정 (Gemma2 모델 사용)
os.environ["OLLAMA_API_BASE"] = "http://localhost:11434"  # Ollama 서버 주소
ollama_model = Ollama(model="gemma2")

C:\Users\human-20\AppData\Local\Temp\ipykernel_9088\2509825096.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  ollama_model = Ollama(model="gemma2")


In [3]:
# 다양한 템플릿 설정
TEMPLATES = {
    "취업": "Based on the following keywords and example, write a personal statement for a job application.",
    "대학원": "Using the provided keywords, draft a personal statement for a graduate school application.",
    "봉사활동": "With the given keywords, write a personal statement emphasizing volunteer experience and motivation."
}

In [4]:
# 언어 지원: 한국어, 영어, 일본어
LANGUAGES = {
    "한국어": "Please write the response in Korean.",
    "영어": "Please write the response in English.",
    "일본어": "Please write the response in Japanese."
}

In [5]:
# 자동 키워드 추천 함수
def recommend_keywords(purpose):
    if purpose == "취업":
        return "책임감, 팀워크, 문제 해결 능력"
    elif purpose == "대학원":
        return "연구 열정, 창의력, 학업 성취도"
    elif purpose == "봉사활동":
        return "사회적 책임감, 희생정신, 리더십"
    else:
        return ""

In [6]:
# 자기소개서 작성 함수
def generate_statement(purpose, language, keywords, example_sentence=None):
    if purpose not in TEMPLATES:
        return "❌ 지원 목적을 올바르게 선택해주세요."
    if language not in LANGUAGES:
        return "❌ 언어를 올바르게 선택해주세요."

    # 템플릿 생성
    template = TEMPLATES[purpose] + "\n\nKeywords: {keywords}\n" + LANGUAGES[language]
    if example_sentence:
        template += f"\n\nExample sentence: {example_sentence}"

    prompt = PromptTemplate(input_variables=["keywords"], template=template)
    chain = LLMChain(llm=ollama_model, prompt=prompt)
    response = chain.run({"keywords": keywords})
    return response

In [7]:
# PDF 저장 함수
def save_to_pdf(statement, filename="personal_statement.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.add_font('MalgunGothic', '', r'C:\Windows\Fonts\malgun.ttf', uni=True)  # '맑은 고딕' 폰트 경로 설정
    pdf.set_font('MalgunGothic', size=12)  # 폰트 설정
    pdf.multi_cell(0, 10, statement)
    pdf.output(filename)
    return f"✔️ PDF 저장 완료: {filename}"

In [9]:
# Gradio 인터페이스
def chatbot_interface(purpose, language, keywords, example_sentence=None, save_pdf=False):
    statement = generate_statement(purpose, language, keywords, example_sentence)
    if save_pdf:
        save_to_pdf(statement)
    return statement

In [10]:
with gr.Blocks() as demo:
    gr.Markdown("# 📝 다목적 자기소개서 작성 도우미")
    gr.Markdown("키워드와 추천 문장을 활용하여 취업, 대학원, 봉사활동 자기소개서를 생성하고 PDF로 저장하세요!")

    # 입력 영역
    with gr.Row():
        purpose_input = gr.Dropdown(label="지원 목적", choices=["취업", "대학원", "봉사활동"], value="취업")
        language_input = gr.Dropdown(label="언어 선택", choices=["한국어", "영어", "일본어"], value="한국어")

    recommended_keywords = gr.Textbox(label="추천 키워드", interactive=False)
    recommend_btn = gr.Button("키워드 추천")
    recommend_btn.click(recommend_keywords, inputs=[purpose_input], outputs=[recommended_keywords])

    with gr.Row():
        keywords_input = gr.Textbox(label="사용자 키워드 입력", placeholder="예: 책임감, 팀워크, 문제 해결 능력")
        example_sentence_input = gr.Textbox(
            label="추천 문장 (선택 사항)",
            placeholder="예: '저는 도전을 두려워하지 않고 성공적으로 프로젝트를 완수했습니다.'"
        )

    save_pdf_toggle = gr.Checkbox(label="PDF로 저장", value=False)

    # 출력 영역
    output = gr.Textbox(label="작성된 자기소개서", lines=6)
    submit_btn = gr.Button("작성하기")
    submit_btn.click(
        fn=chatbot_interface,
        inputs=[purpose_input, language_input, keywords_input, example_sentence_input, save_pdf_toggle],
        outputs=[output]
    )

In [11]:
# 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\human-20\AppData\Local\Temp\ipykernel_9088\4139684956.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=ollama_model, prompt=prompt)
C:\Users\human-20\AppData\Local\Temp\ipykernel_9088\4139684956.py:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"keywords": keywords})
c:\aiproject2\.venv\lib\site-packages\fpdf\ttfonts.py:670: UserWarning: cmap value too big/small: -44022
  warnings.warn("cmap value too big/small: %s" % cm)
c:\aiproject2\.venv\lib\site-packages\fpdf\ttfonts.py:670: UserWarning: cmap value too big/small: -44036
  warnings.warn("cmap value too big/small: %s" % cm)
c:\aiproject2\.venv\lib\site-packages\fpdf\ttfonts.py:670: UserWarning: cmap value too big/small: -44040
  warnings.warn("cmap v